In [1]:
from utils import dump
import json_utils

In [4]:
import os
import io
import sys
import json
from httpretty import httpretty

from rasa_core.train import train_dialogue_model
from rasa_core.agent import Agent
from rasa_core.utils import EndpointConfig, AvailableEndpoints

def test_formbot_example():
    sys.path.append("examples/formbot/")

    p = "examples/formbot/"
    stories = os.path.join(p, "data", "stories.md")
    endpoint = EndpointConfig("https://abc.defg/webhooks/actions")
    endpoints = AvailableEndpoints(action=endpoint)
    agent = train_dialogue_model(os.path.join(p, "domain.yml"),
                                 stories,
                                 os.path.join(p, "models", "dialogue"),
                                 endpoints=endpoints,
                                 policy_config="rasa_core/default_config.yml")
    response = {
        'events': [
            {'event': 'form', 'name': 'restaurant_form', 'timestamp': None},
            {'event': 'slot', 'timestamp': None,
             'name': 'requested_slot', 'value': 'cuisine'}
        ],
        'responses': [
            {'template': 'utter_ask_cuisine'}
        ]
    }

    httpretty.register_uri(
        httpretty.POST,
        'https://abc.defg/webhooks/actions',
        body=json.dumps(response))

    httpretty.enable()

    responses = agent.handle_text("/request_restaurant")

    httpretty.disable()
    b = httpretty.latest_requests[-1].body.decode("utf-8")
    json_utils.write_json_to_file("./dump/form_req_restaurant.json", json.loads(b))

    assert responses[0]['text'] == 'what cuisine?'

    response = {
        "error": "Failed to validate slot cuisine with action restaurant_form",
        "action_name": "restaurant_form"
    }

    httpretty.register_uri(
        httpretty.POST,
        'https://abc.defg/webhooks/actions',
        status=400,
        body=json.dumps(response))

    httpretty.enable()

    responses = agent.handle_text("/chitchat")

    httpretty.disable()
    b = httpretty.latest_requests[-1].body.decode("utf-8")
    json_utils.write_json_to_file("./dump/form_req_chitchat.json", json.loads(b))

    assert responses[0]['text'] == 'chitchat'
    print(responses[0])

test_formbot_example()

Processed Story Blocks: 100%|██████████| 10/10 [00:00<00:00, 111.64it/s, # trackers=5]


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 5, 22)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                7040      
_________________________________________________________________
dense (Dense)                (None, 9)                 297       
_________________________________________________________________
activation (Activation)      (None, 9)                 0         
Total params: 7,337
Trainable params: 7,337
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
66/66 [==============================] - 1s 13ms/step - loss: 2.2056 - acc: 0.1061
Epoch 2/100
66/66 [==============================] - 0s 670us/step - loss: 2.1690 - acc: 0.1970
Epoch 3/100
66/66 [==============================] - 0s 489us/step - loss: 2.1253

66/66 [==============================] - 0s 586us/step - loss: 1.5624 - acc: 0.3939
Epoch 77/100
66/66 [==============================] - 0s 450us/step - loss: 1.5739 - acc: 0.3939
Epoch 78/100
66/66 [==============================] - 0s 582us/step - loss: 1.5607 - acc: 0.3939
Epoch 79/100
66/66 [==============================] - 0s 412us/step - loss: 1.5453 - acc: 0.3939
Epoch 80/100
66/66 [==============================] - 0s 576us/step - loss: 1.5524 - acc: 0.3939
Epoch 81/100
66/66 [==============================] - 0s 602us/step - loss: 1.5248 - acc: 0.3939
Epoch 82/100
66/66 [==============================] - 0s 718us/step - loss: 1.5217 - acc: 0.3939
Epoch 83/100
66/66 [==============================] - 0s 614us/step - loss: 1.5323 - acc: 0.3939
Epoch 84/100
66/66 [==============================] - 0s 678us/step - loss: 1.5102 - acc: 0.3939
Epoch 85/100
66/66 [==============================] - 0s 547us/step - loss: 1.5306 - acc: 0.3939
Epoch 86/100
66/66 [=======================

Processed actions: 66it [00:00, 300.81it/s, # examples=66]


{'recipient_id': 'default', 'text': 'chitchat'}


## train core model

In [18]:
sys.path.append("examples/formbot/")

p = "examples/formbot/"
stories = os.path.join(p, "data", "stories.md")
endpoint = EndpointConfig("http://localhost:5055/webhook")
endpoints = AvailableEndpoints(action=endpoint)
agent = train_dialogue_model(os.path.join(p, "domain.yml"),
                             stories,
                             os.path.join(p, "models", "dialogue"),
                             endpoints=endpoints,
                                 policy_config="rasa_core/default_config.yml")

/Users/xiaofeiwu/miniconda3/envs/bigdata/lib/python3.6/site-packages/pykwalify/core.py:99: UnsafeLoaderWarning: 
The default 'Loader' for 'load(stream)' without further arguments can be unsafe.
Use 'load(stream, Loader=ruamel.yaml.Loader)' explicitly if that is OK.
Alternatively include the following in your code:

  import warnings
  warnings.simplefilter('ignore', ruamel.yaml.error.UnsafeLoaderWarning)

In most other cases you should consider using 'safe_load(stream)'
  data = yaml.load(stream)
Processed Story Blocks: 100%|██████████| 10/10 [00:00<00:00, 125.36it/s, # trackers=5]


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 5, 22)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                7040      
_________________________________________________________________
dense (Dense)                (None, 9)                 297       
_________________________________________________________________
activation (Activation)      (None, 9)                 0         
Total params: 7,337
Trainable params: 7,337
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
66/66 [==============================] - 1s 13ms/step - loss: 2.2246 - acc: 0.0909
Epoch 2/100
66/66 [==============================] - 0s 602us/step - loss: 2.1761 - acc: 0.1515
Epoch 3/100
66/66 [==============================] - 0s 656us/step - loss: 2.1372

Processed actions: 66it [00:00, 287.39it/s, # examples=66]


In [28]:
responses = agent.handle_text("/request_restaurant")
# assert responses[0]['text'] == 'what cuisine?'
print(responses)

responses = agent.handle_text("/chitchat")
# assert responses[0]['text'] == 'chitchat'
print(responses[0])

[{'recipient_id': 'default', 'text': "sorry, I didn't understand you, please try input something else"}]
{'recipient_id': 'default', 'text': 'chitchat'}


In [22]:
strpath=os.path.join(p, "models", "dialogue")
loaded = Agent.load(strpath)
print(loaded.domain.action_names)
print(loaded.domain.intents)
print(loaded.domain.entities)
print(loaded.domain.templates)
print([s.name for s in loaded.domain.slots])

/Users/xiaofeiwu/miniconda3/envs/bigdata/lib/python3.6/site-packages/pykwalify/core.py:99: UnsafeLoaderWarning: 
The default 'Loader' for 'load(stream)' without further arguments can be unsafe.
Use 'load(stream, Loader=ruamel.yaml.Loader)' explicitly if that is OK.
Alternatively include the following in your code:

  import warnings
  warnings.simplefilter('ignore', ruamel.yaml.error.UnsafeLoaderWarning)

In most other cases you should consider using 'safe_load(stream)'
  data = yaml.load(stream)


['action_listen', 'action_restart', 'action_default_fallback', 'action_deactivate_form', 'utter_slots_values', 'utter_noworries', 'utter_chitchat', 'utter_ask_continue', 'restaurant_form']
['affirm', 'chitchat', 'deny', 'inform', 'request_restaurant', 'stop', 'thankyou']
['cuisine', 'num_people', 'number', 'feedback', 'seating']
{'utter_ask_continue': [{'text': 'do you want to continue?'}], 'utter_ask_cuisine': [{'text': 'what cuisine?'}], 'utter_ask_feedback': [{'text': 'please give your feedback'}], 'utter_ask_num_people': [{'text': 'how many people?'}], 'utter_ask_outdoor_seating': [{'text': 'do you want to seat outside?'}], 'utter_ask_preferences': [{'text': 'please provide additional preferences'}], 'utter_chitchat': [{'text': 'chitchat'}], 'utter_default': [{'text': "sorry, I didn't understand you, please try input something else"}], 'utter_noworries': [{'text': 'you are welcome :)'}], 'utter_slots_values': [{'text': 'got slots:\n - cuisine: {cuisine}\n - num_people: {num_people}

In [20]:
print(type(loaded.policy_ensemble))

<class 'rasa_core.policies.ensemble.SimplePolicyEnsemble'>


In [32]:
from rasa_core.interpreter import RegexInterpreter

strpath=os.path.join(p, "models", "dialogue")
endpoint = EndpointConfig("http://localhost:5055/webhook")
endpoints = AvailableEndpoints(action=endpoint)
loaded = Agent.load(strpath, interpreter=RegexInterpreter(), action_endpoint=endpoints.action)

responses = loaded.handle_text("/request_restaurant")
print(responses)
responses = loaded.handle_text("/chitchat")
print(responses)

/Users/xiaofeiwu/miniconda3/envs/bigdata/lib/python3.6/site-packages/pykwalify/core.py:99: UnsafeLoaderWarning: 
The default 'Loader' for 'load(stream)' without further arguments can be unsafe.
Use 'load(stream, Loader=ruamel.yaml.Loader)' explicitly if that is OK.
Alternatively include the following in your code:

  import warnings
  warnings.simplefilter('ignore', ruamel.yaml.error.UnsafeLoaderWarning)

In most other cases you should consider using 'safe_load(stream)'
  data = yaml.load(stream)


[{'recipient_id': 'default', 'text': 'what cuisine?'}]
[{'recipient_id': 'default', 'text': 'chitchat'}, {'recipient_id': 'default', 'text': 'what cuisine?'}]
